In [31]:
import pandas as pd
import gensim
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('msft_tweets - msft_tweets.csv')
df.head()

,date,message,sentiment
0,2014-01-01,"""@BenedictEvans shocking that $MSFT is missing...",2.0
1,2014-01-01,"""RT @ACInvestorBlog: Stocks to Watch for Janua...",1.0
2,2014-01-01,"""Stocks to Watch for January 2, 2014 http:\/\/...",1.0
3,2014-01-01,"""Dow #Stocks Trend $AXP $UTX $CSCO $KO $HD $DI...",0.0
4,2014-01-02,"""Microsoft Corporation : Microsoft Assigned Pa...",1.0


In [12]:
tokenizer  = TweetTokenizer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def clean(message):
    message = message.lower()
    message = re.sub(r"[^A-Za-z0-9]", " ", message)
    message = message.split()
    message = [stemmer.stem(word) for word in message]
    return message

In [13]:
message_list = df['message'].tolist()
cleaned_message_list = [clean(message) for message in message_list]

# build vocabulary and train model
model = gensim.models.Word2Vec(
    cleaned_message_list,
    size=300,
    window=10,
    min_count=1,
    workers=10)
model.train(cleaned_message_list, total_examples=len(cleaned_message_list), epochs=30)

model.save('stockTweetsEmbedding')

In [17]:
embeddedlist = [sum([model.wv.word_vec(word) for word in message]) for message in cleaned_message_list]

In [24]:
labelled_data = embeddedlist[:200] + embeddedlist[300:399]
sentiment_list = df['sentiment'].tolist()
labels = [int(sentiment) for sentiment in (sentiment_list[:200] + sentiment_list[300:399])]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(labelled_data, labels, test_size=0.1)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))

0.5666666666666667
